In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.optim as optim

torch.manual_seed(1)

import torchvision
from torchvision import models
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

import random
random.seed(1)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from PIL import Image
from glob import glob
import os
from os.path import join

class ImageLoader(torch.utils.data.Dataset):
    def __init__(self, path, flag="", transform=None):
        self.transform = transform
        self.data = path
        self.flag = flag
#         /kaggle/input/2021-ai-w10-p2/images/images/4/rangifer_tarandus_s_001241.png
        if self.flag=="train":
            self.label = [int(p.split('/')[-2]) for p in path]
        self.data_len = len(self.data)
        
    def __len__(self):
        return self.data_len
    
    def __getitem__(self, index):
        image = Image.open(self.data[index], mode='r')
        image = image.convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.flag=="train":
#             self.label[index] = self.label[index].to_numpy()
            return image, self.label[index]
        elif self.flag=='test':
            return image
            

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_path = '/kaggle/input/2021-ai-w10-p2/images/images/'
test_path = '/kaggle/input/2021-ai-w10-p2/test_data/test_data'

train_img_path = glob(join(train_path, "*", '*'))
test_img_path = glob(join(test_path, "*"))

In [3]:
test_paths=[]
for i in range(len(test_img_path)):
    test_paths.append(test_path+'/'+str(i)+'.png')

In [4]:
X = ImageLoader(train_img_path, 'train', transform)
test = ImageLoader(test_paths, 'test', transform)

In [5]:
trainloader = torch.utils.data.DataLoader(X, batch_size = 100, shuffle=True, drop_last=True)
testloader = torch.utils.data.DataLoader(test, batch_size = 100, shuffle=False, drop_last=True)

In [6]:
!pip install timm

     |████████████████████████████████| 376 kB 5.0 MB/s            


In [7]:
# import timm
# resnet = timm.create_model(model_name='efficientnet_b0',
#                          pretrained=True, num_classes=10)
# resnet = resnet.to(device)


In [8]:
resnet = models.resnet50(pretrained=True)

# resnet.fc.out_features=10

resnet.layer1.requires_grad=False
resnet.layer2.requires_grad=False
resnet.layer3.requires_grad=False
resnet.layer4.requires_grad=False

linear1 = torch.nn.Linear(1000, 512)
linear2 = torch.nn.Linear(512, 256)
linear3 = torch.nn.Linear(256, 10)

torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear2.weight)

dropout = torch.nn.Dropout(0.2)
relu = torch.nn.ReLU()

resnet = torch.nn.Sequential(resnet, relu, dropout,
                            linear1, relu, dropout,
                            linear2)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [9]:
resnet = resnet.to(device)
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=1e-3, momentum=0.9)
total = len(trainloader)

resnet.train()
for epoch in range(5):
    avgcost = 0
    avgacc = 0
    for X, Y in trainloader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = resnet(X)
        cost = loss(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        predict = torch.argmax(hypothesis, dim=1).cpu()
        cor = predict == Y.cpu()
        acc = cor.sum().item()/len(cor)
        
        avgcost += cost/total
        avgacc += acc/total
    print(epoch, avgcost, avgacc)
        

0 tensor(0.5231, device='cuda:0', grad_fn=<AddBackward0>) 0.8463199999999996
1 tensor(0.1220, device='cuda:0', grad_fn=<AddBackward0>) 0.9605200000000015
2 tensor(0.0601, device='cuda:0', grad_fn=<AddBackward0>) 0.9804399999999989
3 tensor(0.0278, device='cuda:0', grad_fn=<AddBackward0>) 0.9913399999999974
4 tensor(0.0159, device='cuda:0', grad_fn=<AddBackward0>) 0.9949999999999977


In [10]:
sample = pd.read_csv('/kaggle/input/2021-ai-w10-p2/format.csv')

In [11]:
resnet.eval()
idx = 0

with torch.no_grad():
    for X in testloader:
        X = X.to(device)
        predict = torch.argmax(resnet(X), dim=1).cpu().detach().numpy()
        for i in range(predict.shape[0]):
            sample['label'][idx] = predict[i]
            idx += 1


In [12]:
sample

,id,label
0,1,3
1,2,8
2,3,8
3,4,0
4,5,6
...,...,...
9995,9996,8
9996,9997,3
9997,9998,5
9998,9999,1


In [13]:
sample.to_csv('submit.csv', index=False)